In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.linear_model import LogisticRegression
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


from sklearn.impute import SimpleImputer
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/toxic-comment-model/doc2vec-toxic.model
/kaggle/input/toxic-comment-model/doc2vec-toxic.model.docvecs.vectors_docs.npy
/kaggle/input/test-features/Test_features_toxic_comment.npy


In [2]:
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip

Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: test_labels.csv         


In [3]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')
test_labels=pd.read_csv('test_labels.csv')

In [4]:
train_df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [5]:
test_df

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [6]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


### So we can see that a comment can have multiple labels

In [7]:
cols_to_drop=['id']
train_df.drop(cols_to_drop,axis=1,inplace=True)


# Preprocessing Training Data

pre-process each line (tokenize text into individual words, remove punctuation, set to lowercase, etc)

In [8]:
# There might be some missing values so we have to handle them as always
COMMENT = 'comment_text'
train_df[COMMENT].fillna("unknown", inplace=True)
test_df[COMMENT].fillna("unknown", inplace=True)


In [9]:
import gensim

def tokeniser_gen(df,tokens_only=False):
   
    for i,iterator in enumerate(range(len(df))):
        line=df.loc[iterator]['comment_text']
        tokens=gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

        
train_corpus=list(tokeniser_gen(train_df))

test_corpus = list(tokeniser_gen(test_df, tokens_only=True))

refer [here](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html)

In [10]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=50)

## Build Vocabulary
Essentially, the vocabulary is a dictionary (accessible via model.wv.vocab) of all of the unique words extracted from the training corpus along with the count 

(e.g., model.wv.vocab['penalty'].count for counts for the word penalty).


In [11]:
model.build_vocab(train_corpus)

In [12]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=10)

In [13]:
model.save("/kaggle/working/doc2vec-toxic-100.model")

In [14]:
from gensim.models.doc2vec import Doc2Vec

model_df = Doc2Vec.load("/kaggle/working/doc2vec-toxic-100.model")
vector = model_df.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[-0.00026192  0.0247625  -0.04677513 -0.03049093  0.00798524  0.05021098
  0.01348796  0.0634729   0.02222658 -0.05092812 -0.06360117  0.00370967
 -0.04261972 -0.00251006  0.04424752 -0.06525391  0.00286783 -0.05631689
  0.00702617  0.02948709  0.05153012  0.05731047 -0.14501384 -0.11308805
  0.09189925  0.01026158  0.01392639 -0.00034943 -0.03084719 -0.0343532
  0.0914257   0.08337139 -0.14861494  0.22628593  0.07403352 -0.01998601
  0.08459276 -0.00375962  0.10610422  0.12297155 -0.00825975 -0.00082397
 -0.00529838 -0.04928123 -0.04579074  0.02327015 -0.02610426 -0.11656953
  0.09049954 -0.1305277   0.03517422  0.05614713  0.00462316 -0.06297098
  0.07033511  0.06028103 -0.01410527  0.04165618  0.01875094 -0.05955436
  0.01693773  0.07279351  0.04817581  0.00926701  0.15096577 -0.03808413
 -0.02743369  0.04082724 -0.01457325 -0.01788765  0.0613092   0.04229861
  0.07595947 -0.00593287  0.03732304 -0.02287145 -0.05347645  0.04444833
  0.12764192  0.08956876 -0.04809857  0.09381608 -0.

In [15]:
empty_test_fea=[]
for i in range(len(test_corpus[:])):
    vector=model_df.infer_vector(test_corpus[i])
    empty_test_fea.append(vector)
#     print(i)

In [16]:
from numpy import save

In [17]:

test_features=empty_test_fea[:]
test_features=np.array(test_features)
test_features.shape
np.save('/kaggle/working/Test_features_toxic_comment',test_features)


### Loading training and testing Features

In [18]:
train_features=np.load('/kaggle/working/doc2vec-toxic-100.model.docvecs.vectors_docs.npy')
test_features=np.load('/kaggle/working/Test_features_toxic_comment.npy')
print(test_features.shape)
print(train_features.shape)

(153164, 100)
(159571, 100)


Basic Naive Bayes Equation

In [19]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [20]:
x = train_features
test_x = test_features

In [21]:
from scipy import sparse
x = sparse.csr_matrix(x)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(x)


SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

Fit a model for one dependent at a time:

In [22]:
def get_mdl(y):
    y = y.values
    r=pr(1,y) / pr(0,y)
#     r = np.log(pr(1,y) / pr(0,y))#ignored values were going to negative  infinity so instead  directly took ratio
# becuase  actually we  just want ratio 
    
    return r

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((len(test_df), len(label_cols)))


scores=[]
for i, j in enumerate(label_cols):
    print('fit', j)
    m = LogisticRegression(C=4,max_iter=2000)
    r = get_mdl(train_df[j])
    x_nb = x.multiply(r)
    y=train_df[j]
    cv_score = np.mean(cross_val_score(m, x_nb, y, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(j, cv_score))
    m.fit(x_nb, y)
    preds[:,i] = m.predict_proba(np.multiply(test_x,r))[:,1]
print('Total CV score is {}'.format(np.mean(scores)))



fit toxic


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


CV score for class toxic is 0.8746171514398359


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


fit severe_toxic
CV score for class severe_toxic is 0.8426826446716434
fit obscene
CV score for class obscene is 0.8900476565384459
fit threat
CV score for class threat is 0.852271707587117
fit insult
CV score for class insult is 0.8823941122662068
fit identity_hate
CV score for class identity_hate is 0.8780029085287673
Total CV score is 0.8700026968386694


In [24]:
submid = pd.DataFrame({'id': submission["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)

#  Not for this kernel


In [25]:
# train_df
# clean=train_df.loc[(train_df.toxic==0) & (train_df.severe_toxic==0) & (train_df.obscene==0)&(train_df.threat==0)&(train_df.insult==0)&(train_df.identity_hate==0)]
# toxic=train_df.loc[(train_df.toxic==1) ]
# severe_toxic=train_df.loc[ (train_df.severe_toxic==1)]
# obscene=train_df.loc[ (train_df.obscene==1)]
# threat=train_df.loc[(train_df.threat==1)]
# insult=train_df.loc[(train_df.insult==1)]
# identity_hate=train_df.loc[(train_df.identity_hate==1)]
# print(clean.shape,toxic.shape,severe_toxic.shape,obscene.shape,threat.shape,insult.shape,identity_hate.shape)

In [26]:
# clean=clean.loc[:,'comment_text'].to_frame()
# toxic=toxic.loc[:,'comment_text'].to_frame()
# severe_toxic=severe_toxic.loc[:,'comment_text'].to_frame()
# obscene=obscene.loc[:,'comment_text'].to_frame()
# threat=threat.loc[:,'comment_text'].to_frame()
# insult=insult.loc[:,'comment_text'].to_frame()
# identity_hate=identity_hate.loc[:,'comment_text'].to_frame()
# print(clean.shape,toxic.shape,severe_toxic.shape,obscene.shape,threat.shape,insult.shape,identity_hate.shape)

In [27]:
# clean['label']=[0]*len(clean)
# toxic['label']=[1]*len(toxic)
# severe_toxic['label']=[2]*len(severe_toxic)
# obscene['label']=[3]*len(obscene)
# threat['label']=[4]*len(threat)
# insult['label']=[5]*len(insult)
# identity_hate['label']=[6]*len(identity_hate)
# print(clean.shape,toxic.shape,severe_toxic.shape,obscene.shape,threat.shape,insult.shape,identity_hate.shape)

In [28]:

# clean=clean.append(toxic,ignore_index=True)
# clean=clean.append(severe_toxic,ignore_index=True)
# clean=clean.append(obscene,ignore_index=True)
# clean=clean.append(threat,ignore_index=True)
# clean=clean.append(insult,ignore_index=True)
# trainer=clean.append(identity_hate,ignore_index=True)


In [29]:
# trainer

In [30]:
# # There might be some missing values so we have to handle them as always
# COMMENT = 'comment_text'
# trainer[COMMENT].fillna("unknown", inplace=True)
# test_df[COMMENT].fillna("unknown", inplace=True)

In [31]:
# import gensim

# def tokeniser_gen(df,tokens_only=False):
   
#     for i,iterator in enumerate(range(len(df))):
#         line=df.loc[iterator]['comment_text']
#         tokens=gensim.utils.simple_preprocess(line)
#         if tokens_only:
#             yield tokens # for testing we are only creting tokens of a sentence not tagging them
#         else:
#             #for training only
#             yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
#             #for training doc2vec we need to tag documents here we are just tagging them by integers

        
# train_corpus_new=list(tokeniser_gen(trainer))

# test_corpus_new = list(tokeniser_gen(test_df, tokens_only=True))


In [32]:
model_new = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=5)

In [33]:
# model_new.build_vocab(train_corpus_new)
# model_new.train(train_corpus, total_examples=model_new.corpus_count, epochs=model_new.epochs)

In [34]:
# model_new.save("/kaggle/working/doc2vec-toxic-75-new.model")
# from gensim.models.doc2vec import Doc2Vec

# model_df_new = Doc2Vec.load("/kaggle/working/doc2vec-toxic-75-new.model")
# vector = model_df_new.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
# print(vector)

In [35]:
# empty_test_fea=[]
# for i in range(len(test_corpus[:5])):
#     vector=model_df_new.infer_vector(test_corpus[i])
#     empty_test_fea.append(vector)
# #     print(i)

# test_features=empty_test_fea[:]
# test_features=np.array(test_features)
# test_features.shape
# np.save('/kaggle/working/Test_features_toxic_comment',test_features)


In [36]:
# train_features=np.load('/kaggle/working/doc2vec-toxic-75-new.model.docvecs.vectors_docs.npy')
# test_features=np.load('/kaggle/working/Test_features_toxic_comment.npy')
# print(test_features.shape)
# print(train_features.shape)

In [37]:
# def pr(y_i, y):
#     p = x[y==y_i].sum(0)
#     return (p+1) / ((y==y_i).sum()+1)

# x = train_features

# test_x = test_features

# from scipy import sparse
# x = sparse.csr_matrix(x)
# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp.fit(x)


In [38]:
# test_x[4:].shape

In [39]:
# from sklearn.linear_model import LogisticRegression

In [40]:
# d=clf.predict_proba(test_x[4:])

# print(d)



In [41]:
# max_index_col = np.argmax(d, axis=1)
# max_index_col

In [42]:
# print('sahi')

In [43]:
# from sklearn.model_selection import cross_val_score
# ls=[]
# clf = LogisticRegression(solver='sag',max_iter=1000)
# cv_score = ls.append(cross_val_score(clf, x, trainer['label'], cv=10, scoring='roc_auc'))
# clf.fit(x, trainer['label'])


In [44]:
# def get_mdl(y):
#     y = y.values
#     r=pr(1,y) / pr(0,y)
# #     r = np.log(pr(1,y) / pr(0,y))#ignored values were going to negative  infinity so instead  directly took ratio
# # becuase  actually we  just want ratio 
#     m = LogisticRegression(C=4,max_iter=1000)
    
#     x_nb = x.multiply(r)
#     return m.fit(x_nb, y), r

# from sklearn.linear_model import LogisticRegression
# label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# preds = np.zeros((len(test_df), len(label_cols)))



# for i, j in enumerate(label_cols):
#     print('fit', j)
#     m,r = get_mdl(train_df[j])
#     preds[:,i] = m.predict_proba(np.multiply(test_x,r))[:,1]

In [45]:
# f="he  is fat"
# f=gensim.utils.simple_preprocess(f)
# m1 = Doc2Vec.load("/kaggle/working/doc2vec-toxic-75.model")
# # vector1 = m1.infer_vector(f)
# empty_test_fea1=[]
# for i in range(1):
#     vector1=m1.infer_vector(f)
#     empty_test_fea1.append(vector1)
#     print(i)
# test_features1=empty_test_fea1[:]
# test_features1=np.array(test_features1)
# test_features1.shape


In [46]:
# from sklearn.linear_model import LogisticRegression
# label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# preds1 = np.zeros((1, len(label_cols)))



# for i, j in enumerate(label_cols):
#     print('fit', j)
#     m,r = get_mdl(train_df[j])
#     preds1[:,i] = m.predict_proba(np.multiply(test_features1,r))[:,1]

In [47]:
# submid1 = pd.DataFrame({'id': submission["id"]})
# submission1 = pd.concat([submid1, pd.DataFrame(preds1, columns = label_cols)], axis=1)
# submission1.to_csv('submission1.csv', index=False)